# DS 3010: Building the Visualization

After applying DBScan, the team is interested in using the results to create an interactive map using Folium, a Python library that uses the Leaflet.js to visualize data on a map. After the map is creating, it will be ported to a HTML file that will be used for the presentation.


In [ ]:
!pip install folium

In [ ]:
import pandas as pd

# Unpack restaurant data into csv
restaurants = pd.read_csv('restaurants.csv')

restaurants.tail(5)

,name,postal_code,latitude,longitude,stars,review_count,categories,performance_cluster,geospatial_cluster
3522,Stina Pizzeria,19145,39.925205,-75.174728,4.5,112,"Pizza, Restaurants, Mediterranean",1,0
3523,Flip-N-Pizza,19123,39.970720,-75.157752,4.0,16,"Restaurants, American (Traditional), Chicken W...",0,0
3524,Grilly Cheese,19104,39.957447,-75.189916,3.0,5,"Specialty Food, Event Planning & Services, Res...",3,0
3525,PrimoHoagies,19128,40.032483,-75.214430,3.0,55,"Restaurants, Specialty Food, Food, Sandwiches,...",3,5
3526,Adelita Taqueria & Restaurant,19147,39.935982,-75.158665,4.5,35,"Restaurants, Mexican",1,0


In [78]:
import colorcet as cc
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from matplotlib.colors import to_hex

def draw_clusters(map, df):
  # Define overlay for the clusters
  cluster_overlay = folium.FeatureGroup(name="Density Zones")

  # Group by geospatial cluster
  groups = df.groupby('geospatial_cluster')

  # Get colors
  colormap = plt.get_cmap('cet_glasbey_category10', len(groups))

  print(f'Number of Clusters: {len(groups)}')

  # Draw nodes for each restaurant, giving a different color to each cluster
  for index, (group_name, group_df) in enumerate(groups):

    # Create a marker cluster for the geospatial cluster
    marker_cluster = MarkerCluster().add_to(cluster_overlay)

    # Add each restaurant of the geospatial cluster to the marker cluster
    for _, restaurant in group_df.iterrows():
      folium.CircleMarker(
        location=[restaurant['latitude'], restaurant['longitude']],
        radius=4,
        color=to_hex(colormap(index)),
        opacity=0,
        fill=True,
        fill_opacity=0.8,
        tooltip=restaurant['name'],
        popup=f"Stars: {restaurant['stars']}"
      ).add_to(marker_cluster)

  return cluster_overlay

def draw_nodes(map, df):
  # Define overlay for the clusters
  nodes_overlay = folium.FeatureGroup(name="Locations")

  # Group by geospatial cluster
  groups = df.groupby('geospatial_cluster')

  # Get colors
  colormap = plt.get_cmap('cet_glasbey_category10', len(groups))

  # Draw nodes for each restaurant, giving a different color to each cluster
  for index, (group_name, group_df) in enumerate(groups):
    # Add each restaurant of the geospatial cluster to the marker cluster
    for _, restaurant in group_df.iterrows():
      folium.CircleMarker(
        location=[restaurant['latitude'], restaurant['longitude']],
        radius=4,
        color=to_hex(colormap(index)),
        opacity=0,
        fill=True,
        fill_opacity=0.8,
        tooltip=restaurant['name'],
        popup=f"Stars: {restaurant['stars']}"
      ).add_to(nodes_overlay)

  return nodes_overlay


# Starting coordinates
start_latitude = restaurants['latitude'].mean()
start_longitude = restaurants['longitude'].mean()


# Add base map
map = folium.Map(location=(start_latitude, start_longitude))

# Add overlays
cluster_overlay = draw_clusters(map, restaurants)
cluster_overlay.add_to(map)
nodes_overlay = draw_nodes(map, restaurants)
nodes_overlay.add_to(map)

# Add controls to display layers
folium.LayerControl().add_to(map)

map

Output hidden; open in https://colab.research.google.com to view.

In [79]:
map.save('index.html')